# Page for trying out the data

In [1]:
import pandas as pd
import time, os, shutil
import nltk
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN


W0510 11:12:24.327456  5180 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [ ]:
#df1 = pd.read_csv('amazon_reviews_us_Mobile_Electronics_v1_00.tsv',sep="\t", error_bad_lines=False)
# bad lines exist......
#data_source = 'amazon_reviews_us_Mobile_Electronics_v1_00.tsv'
data_source='amazon_reviews_us_PC_v1_00.tsv.gz'
#data_source='amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
df1 = pd.read_csv(data_source,sep="\t", error_bad_lines=False)

### DATA COLUMNS:
marketplace       - 2 letter country code of the marketplace where the review was written.  
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  
review_id         - The unique ID of the review.  
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.  
product_parent    - Random identifier that can be used to aggregate reviews for the same product.  
product_title     - Title of the product.  
product_category  - Broad product category that can be used to group reviews  
                    (also used to group the dataset into coherent parts).  
star_rating       - The 1-5 star rating of the review.  
helpful_votes     - Number of helpful votes.  
total_votes       - Number of total votes the review received.  
vine              - Review was written as part of the Vine program.  
verified_purchase - The review is on a verified purchase.  
review_headline   - The title of the review.  
review_body       - The review text.  
review_date       - The date the review was written.  

Filter out the ones not versified and get a subset of the data

In [ ]:
df1 = df1.loc[df1['verified_purchase']=='Y',['product_title', 'helpful_votes','review_headline', 'review_body']]

In [ ]:
df2 = df1.loc[:,['review_headline', 'review_body']]

In [ ]:
def clean_html(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

In [ ]:
df2['summary'] = df2.review_headline
df2['text'] = df2.review_body
df2 = df2.loc[:,['text','summary']].reset_index(drop='True')

In [ ]:
df2 = pd.DataFrame(df2)

In [ ]:
df2[:5]

In [ ]:
df2['c_text'] = df2['text'].apply(lambda x: clean_html(str(x)))

In [ ]:
df2['c_summary'] = df2['summary'].apply(lambda x: clean_html(str(x)))

In [ ]:
df2 = df2.loc[:,['c_text','c_summary']]

In [ ]:
df2['text'] = df2['c_text']
df2['summary'] = df2['c_summary']
df2 = df2.loc[:,['text','summary']]

In [ ]:
df2['len_text'] = df2['text'].apply(lambda x: len(x.split()))
df2['len_summary'] = df2['summary'].apply(lambda x: len(x.split()))
df2[:10]

In [ ]:
df2.to_csv('output/sub-set/origin_PC.csv',index=False)

In [ ]:
df2.len_text.describe(include="all").apply(lambda x: format(x, 'f'))

In [ ]:
df2.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

In [ ]:
df3 = df2[df2.len_summary>=10].reset_index(drop=True)

In [ ]:
df3 = df3[df3.len_text>=50].reset_index(drop=True)

In [ ]:
df3 = df3[df3.len_text<=300].reset_index(drop=True)

In [ ]:
df3.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

In [ ]:
df3.len_text.describe(include="all").apply(lambda x: format(x, 'f'))

In [ ]:
df3.to_csv('output/sub-set/word_lim_50_300_PC.csv',index=False)

Reload and start from here

In [2]:
df = pd.read_csv('output/sub-set/word_lim_50_300_PC.csv')

In [5]:
df.len_text.describe(include="all").apply(lambda x: format(x, 'f'))                                                          

count    216264.000000
mean        109.970087
std          58.410543
min          50.000000
25%          65.000000
50%          90.000000
75%         138.000000
max         300.000000
Name: len_text, dtype: object

In [6]:
df.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

count    216264.000000
mean         12.845545
std           3.689135
min          10.000000
25%          10.000000
50%          12.000000
75%          14.000000
max         615.000000
Name: len_summary, dtype: object

In [20]:
from bs4 import BeautifulSoup
from html import unescape

def clean_text(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) ## remove URL
    text = BeautifulSoup(unescape(text), 'lxml').text ## remove strange characters like &amp; 
    text = re.sub(r'[_"\-%()|+&=*%#@\[\]/\r\n\t]', '', text) ## remove other unwanted characters
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text) ## if more than 2 consecutive e or o, remove them
    text = re.sub(r"([^eoEO])\1\1+",r"\1",one) ## if more than 2 consecutive other characters, remove them
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text

In [21]:
df['text'] = df['text'].apply(lambda x: clean_text(x))

In [23]:
df['summary'] = df['summary'].apply(lambda x: clean_text(x))

In [24]:
df['len_text'] = df['text'].apply(lambda x: len(x.split()))
df['len_summary'] = df['summary'].apply(lambda x: len(x.split()))

In [25]:
df.len_text.describe(include="all").apply(lambda x: format(x, 'f'))   

count    216264.000000
mean        109.649567
std          58.200438
min           2.000000
25%          65.000000
50%          90.000000
75%         137.000000
max         300.000000
Name: len_text, dtype: object

In [26]:
df.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))   

count    216264.000000
mean         12.743388
std           3.509147
min           2.000000
25%          10.000000
50%          12.000000
75%          14.000000
max         435.000000
Name: len_summary, dtype: object

In [30]:
df = df.loc[df.len_summary>=10]
df = df.loc[df.len_text>=50]
df = df.loc[df.len_summary<=20]
df = df.loc[df.len_text<=200]

In [31]:
df = df.reset_index(drop=True)
df

,text,summary,len_text,len_summary
0,Love that Finite makes different designs and t...,Great choices of covers but no Autoshut off or on,58,10
1,I had to make a few modifications as the hole ...,I had to make a few modifications as the hole .,78,11
2,Fantastic battery for all applications! I use ...,Great customer service from Anker! Battery hol...,150,11
3,"It is as described, WIn8 mini pc. However, you...",OK but over heats I'd spend more and get a mor...,156,14
4,I live in a small house with few walls. Signal...,I live in a small house with few walls. .,60,10
5,As an electrical engineer I bought this for th...,If you are just trying to use the head like I ...,114,16
6,I own an Alienware mx 18 r2 and I have been hu...,Single greatest backpack I uave ever owned for...,67,10
7,Although the description says it must be wired...,Only Works On The Same Circuit As The ModemI T...,55,10
8,I bought this case to protect my surface pro. ...,"It's great, if you don't need it to provide an...",158,13
9,Ok so this rating is specifically the result o...,NO STREAMING VIDEO ON CHILDREN'S ACCOUNTS WITH...,97,12


In [32]:
df.to_csv('output/amazon/clean_PC1.csv',index=False)

In [ ]:
##https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/blob/master/en
